<a href="https://colab.research.google.com/github/Ksenyff/AI/blob/main/Design_of_simulation_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Документы:
- [пример плана эксперимента](https://docs.google.com/document/d/1z6WkgkI2ZFticx6dgWjyOvmX3FjtCme7/edit?usp=sharing&ouid=104512700084689419560&rtpof=true&sd=true)
- [документ планирования эксперимента](https://docs.google.com/document/d/14sjerL5Hxy1s9UxwpiUPVhPma7mAZtS2kGt7Xg4zcOs/edit?usp=sharing)

# Построение матрицы планирования полного факторного эксперимента

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install pydoe3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [ ]:
from pyDOE3 import *

In [ ]:
def create_full_factorial_design(factors, normalize=True, append_initial_levels=True, randomize=False):

  # отдельно зададим количество уровней каждого фактора
  factors_levels = [len(factors[factor]) for factor in factors.keys()]

  # рассчитаем матрицу полного факторного планирования
  matrix = fullfact(factors_levels)

  if normalize:
    matrix = normalize_matrix(factors, matrix)
  keys = np.array(list(factors.keys()))

  if append_initial_levels:
    factor_matrix = append_factor_levels(factors, matrix)
    matrix = np.concatenate((matrix, factor_matrix), axis=1)
    keys = np.concat((np.array(list(factors.keys())), np.array(list(factors.keys())) + "_"))

  df = pd.DataFrame(matrix, columns=keys)

  if randomize:
      df = df.sample(frac=1).reset_index(drop=True)

  return df

def append_factor_levels(factors, matrix):

  factor_levels_matrix = matrix.copy()
  # для каждого столбца (фактора)
  for i, (factor, value) in enumerate(factors.items()):

    # возьмем вычисленные уровни фактора
    ffd_levels = np.array(np.unique(factor_levels_matrix[:,i])).reshape(-1)

    # возьмем наши изначальные уровни
    nonscaled_levels = np.array(np.unique(np.array(factors[factor]))).reshape(-1)

    # для удобства зададим словарь соответствия вычисленного уровня фактора нормализованному
    replacement_dict = {level: scaled_level for level, scaled_level in zip(ffd_levels, nonscaled_levels)}

    # поменяем уровень фактора на нормализованный в заданном столбце i
    factor_levels_matrix[:,i] = np.array([replacement_dict.get(x) for x in factor_levels_matrix[:,i]])

  return factor_levels_matrix

def normalize_matrix(factors, matrix):
  # пронормализуем каждый столблец (фактор) матрицы полного факторного планирования в диапазоне [-1, 1]
  scaler = MinMaxScaler(feature_range=(-1, 1))

  # для каждого столбца (фактора)
  for i, (factor, value) in enumerate(factors.items()):

    # возьмем вычисленные уровни фактора
    ffd_levels = np.array(np.unique(matrix[:,i])).reshape(-1)

    # возьмем наши изначальные уровни
    nonscaled_levels = np.array(np.unique(np.array(factors[factor]))).reshape(-1)

    # вычислим соответствующие изначальным уровням нормализованные значения
    scaled_levels = scaler.fit_transform(nonscaled_levels.reshape(-1, 1)).reshape(-1)

    # для удобства зададим словарь соответствия вычисленного уровня фактора нормализованному
    replacement_dict = {level: scaled_level for level, scaled_level in zip(ffd_levels, scaled_levels)}

    # поменяем уровень фактора на нормализованный в заданном столбце i
    matrix[:,i] = np.array([replacement_dict.get(x) for x in matrix[:,i]])

  return matrix

In [ ]:
def encode_category(factors, factor):
  encoder = LabelEncoder()
  for factor_name in [factor]:
    factors[factor_name] = np.sort(encoder.fit_transform(factors[factor_name]))
  return factors

In [ ]:
factors = {
    "planing hor":[1.0, 2.0, 3.0, 4.0, 5.0],
    "width":[0.1, 0.2, 0.3, 0.4, 0.5]
}